# Loading MIMIC III Dataset 

In [ ]:
!  python -m pip install "dask[dataframe]" --upgrade 

Requirement already up-to-date: dask[dataframe] in /opt/venv/lib/python3.7/site-packages (2.30.0)
     |████████████████████████████████| 91 kB 9.0 MB/s 
     |████████████████████████████████| 55 kB 5.9 MB/s 
  Created wheel for locket: filename=locket-0.2.0-py3-none-any.whl size=4040 sha256=2313b2babfb4d4a928340d9b227f631d823513bb5307d1620d00762f0026500f
  Stored in directory: /home/jovyan/.cache/pip/wheels/7d/21/ce/56f01c644a11bde5d09ecae16d9b5e9d7e988187624fd28fec
Successfully built locket
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
! python -m pip install "dask[array]" --upgrade

Requirement already up-to-date: dask[array] in /opt/venv/lib/python3.7/site-packages (2.30.0)
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import dask.dataframe as dd
pd.options.display.float_format = '{:,}'.format

ModuleNotFoundError: No module named 'dask'

## Not included in the Working Data Table 
 
We decided to not include the data into the working frame because the information was not relevant to answer the prediction problem. 

* CALLOUT
* CAREGIVERS
* NOTEEVENTS

# Tables containing Patient Data

As all dates in MIMIC-III are anonymized to protect patient confidentiality, all dates in this table have been shifted. Note that the chronology for an individual patient has been unaffected however, and quantities such as the difference between two dates remain true to reality.

## Admissions
The ADMISSIONS table gives information regarding a patient’s admission to the hospital. Since each unique hospital visit for a patient is assigned a unique HADM_ID.
Information available includes timing information for admission and discharge, demographic information, the source of the admission, whether the patient died within the given hospitalization. In addition the table contains a preliminary, free text diagnosis for the patient on hospital admission.

In [ ]:
df_admissions = pd.read_csv('data/ADMISSIONS.csv.gz', compression='gzip')

In [ ]:
df_admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [ ]:
df_admissions_newborn = df_admissions[df_admissions.ADMISSION_TYPE == "NEWBORN"]
df_admissions_newborn.shape

NameError: name 'df_admissions' is not defined

In [ ]:
df_admissions_adults = df_admissions[df_admissions.ADMISSION_TYPE != "NEWBORN"]

In [ ]:
df_admissions_adults.shape

(51113, 19)

In [ ]:
liste_patienten_adult = list(df_admissions_adults.HADM_ID.unique())
len(liste_patienten_adult)

51113

In [ ]:
df_admissions.shape

(58976, 19)

58976 Admissions and 19 columns

In [ ]:
df_admissions.HADM_ID.nunique()

58976

genauso viele verschiedene Admission-IDs wie Zeilen. Also keine Duplikate.

In [ ]:
df_admissions.SUBJECT_ID.nunique()

46520

Insgesamt 46520 Patienten, d.h. einige Patienten wurden mehrmals ins Krankenhaus eingeliefert.

In [ ]:
df_admissions_del=df_admissions.dropna(subset=["DIAGNOSIS"])

In [ ]:
df_admissions_del[df_admissions_del.DIAGNOSIS.str.contains("RESPIRATORY")]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
106,104,101,175533,2196-09-26 18:36:00,2196-10-12 13:17:00,2196-10-12 13:17:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,NaN,MARRIED,ASIAN,2196-09-26 12:50:00,2196-09-26 18:37:00,RESPIRATORY FAILURE,1,1
360,630,505,116719,2154-03-30 14:59:00,2154-04-01 16:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Private,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,ACUTE MENTAL STATUS CHANGES/RESPIRATORY DISTRESS,0,1
613,1100,886,130937,2139-03-08 10:47:00,2139-03-09 17:05:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,NaN,NaN,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,RESPIRATORY FAILURE,0,1
650,1137,914,124723,2178-02-26 12:14:00,2178-03-13 15:30:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LEFT AGAINST MEDICAL ADVI,Medicare,NaN,CATHOLIC,SINGLE,WHITE,NaN,NaN,HYPOTENSION;RESPIRATORY DECOMPRESSION;S/P LIVE...,0,1
682,835,679,156345,2141-10-19 14:58:00,2141-10-24 11:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,JEWISH,MARRIED,WHITE,2141-10-19 10:21:00,2141-10-19 15:08:00,ACUTE RESPIRATORY FAILURE,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58428,58861,99613,177517,2153-02-10 01:00:00,2153-02-26 17:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,ENGL,CATHOLIC,DIVORCED,WHITE,2153-02-09 23:59:00,2153-02-10 02:56:00,RESPIRATORY FAILURE;PNEUMONIA;CHRONIC OBST PUL...,0,1
58623,58278,97772,119334,2124-07-18 23:39:00,2124-07-19 12:30:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SHORT TERM HOSPITAL,Medicare,PTUN,PROTESTANT QUAKER,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,RESPIRATORY FAILURE,0,1
58814,56986,93814,134288,2194-07-22 14:42:00,2194-07-24 17:30:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Private,ENGL,UNOBTAINABLE,NaN,UNABLE TO OBTAIN,NaN,NaN,CHRONIC RESPIRATORY FAILURE; TRAC OBSTRUCTED A...,0,1
58829,57758,96229,195294,2143-02-09 13:24:00,2143-02-10 07:52:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LEFT AGAINST MEDICAL ADVI,Private,ENGL,CATHOLIC,SINGLE,WHITE,2143-02-09 12:41:00,2143-02-09 15:00:00,RESPIRATORY ARREST,0,1


Future Work:
* Duplicates 
* ADMITTIME / DISCHTIME - calculate LOS 
* Ectract all ADMISSION TYPE expect NEWBORN 
* Drop: Admission Location, Discharge Location, Insurance, Language, Religion, Marietal Status, Ethnicity, EDREGTIME, EDOUTTIME, DIAGNOSIS

## CPTevents
Procedures recorded as Current Procedural Terminology (CPT) codes.

In [ ]:
dd_cpt = dd.read_csv('data/CPTEVENTS.csv.gz', compression='gzip')

AttributeError: module 'dask' has no attribute 'read_csv'

In [ ]:
df_cpt.head()

,ROW_ID,SUBJECT_ID,HADM_ID,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION
0,317,11743,129545,ICU,NaN,99232,"99,232.0",NaN,6.0,Evaluation and management,Hospital inpatient services,NaN
1,318,11743,129545,ICU,NaN,99232,"99,232.0",NaN,7.0,Evaluation and management,Hospital inpatient services,NaN
2,319,11743,129545,ICU,NaN,99232,"99,232.0",NaN,8.0,Evaluation and management,Hospital inpatient services,NaN
3,320,11743,129545,ICU,NaN,99232,"99,232.0",NaN,9.0,Evaluation and management,Hospital inpatient services,NaN
4,321,6185,183725,ICU,NaN,99223,"99,223.0",NaN,1.0,Evaluation and management,Hospital inpatient services,NaN


In [ ]:
df_cpt.shape

(573146, 12)

In [ ]:
df_cpt.HADM_ID.isnull().sum()

0

## DATETIMEEVENTS
DATETIMEEVENTS contains all date measurements about a patient in the ICU. For example, the date of last dialysis would be in the DATETIMEEVENTS table, but the systolic blood pressure would be in the table CHARTEVENTS.   
Wir hoffen, in dieser Tabelle die Information darüber, wann ein Patient intubiert und extubiert wurde, zu erhalten.


In [ ]:
df_datetimeevents = pd.read_csv('data/DATETIMEEVENTS.csv.gz', 
                                compression='gzip', dtype={'HADM_ID': 'float64','ICUSTAY_ID': 'float64'})

/opt/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_datetimeevents.head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,711,7657,"121,183.0","297,945.0",3411,2172-03-14 11:00:00,2172-03-14 11:52:00,16446,NaN,Date,nan,nan,nan,NotStopd
1,712,7657,"121,183.0","297,945.0",3411,2172-03-14 13:00:00,2172-03-14 12:36:00,16446,NaN,Date,nan,nan,nan,NotStopd
2,713,7657,"121,183.0","297,945.0",3411,2172-03-14 15:00:00,2172-03-14 15:10:00,14957,NaN,Date,nan,nan,nan,NotStopd
3,714,7657,"121,183.0","297,945.0",3411,2172-03-14 17:00:00,2172-03-14 17:01:00,16446,NaN,Date,nan,nan,nan,NotStopd
4,715,7657,"121,183.0","297,945.0",3411,2172-03-14 19:00:00,2172-03-14 19:29:00,14815,NaN,Date,nan,nan,nan,NotStopd
5,716,7657,"121,183.0","297,945.0",3411,2172-03-14 21:30:00,2172-03-14 21:43:00,14815,NaN,Date,nan,nan,nan,NotStopd
6,717,7657,"121,183.0","297,945.0",3411,2172-03-15 00:00:00,2172-03-15 00:27:00,14815,NaN,Date,nan,nan,nan,NotStopd
7,718,7657,"121,183.0","297,945.0",3411,2172-03-15 01:00:00,2172-03-15 01:44:00,14815,NaN,Date,nan,nan,nan,NotStopd
8,719,7657,"121,183.0","297,945.0",3411,2172-03-15 03:30:00,2172-03-15 03:48:00,14815,NaN,Date,nan,nan,nan,NotStopd
9,720,7657,"121,183.0","297,945.0",3411,2172-03-15 05:00:00,2172-03-15 04:58:00,14815,NaN,Date,nan,nan,nan,NotStopd


In [ ]:
df_datetimeevents.shape

(4485937, 14)

In [ ]:
df_datetimeevents.HADM_ID.isnull().sum()

1772

Auf diese Datetimeevents verzichten wir.

## Diagnosis_ICD
SEQ_NUM provides the order in which the ICD diagnoses relate to the patient. ICD diagnoses are ordered by priority - and the order does have an impact on the reimbursement for treatment.

In [ ]:
df_diagnosis = pd.read_csv('data/DIAGNOSES_ICD.csv.gz', 
                                compression='gzip')

In [ ]:
df_diagnosis.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [ ]:
df_diagnosis.shape

(651047, 5)

In [ ]:
df_diagnosis.HADM_ID.isnull().sum()

0

In [ ]:
df_diagnosis[df_diagnosis.ICD9_CODE == '51881'].SUBJECT_ID.nunique()

6719

In [ ]:
df_diagnosis[df_diagnosis.ICD9_CODE == '51881'].HADM_ID.nunique()

7497

Es gibt Patienten, die mehrmals mit der Diagnose "ARDS" im Krankenhaus waren.

In [ ]:
# DF with all rows with diagnosis ARDS
df_patienten = df_diagnosis[df_diagnosis.ICD9_CODE == '51881']

In [ ]:
liste_patienten = list(df_patienten.HADM_ID.unique())
len(liste_patienten)


7497

In [ ]:
import csv
with open('data/liste_patienten.csv', 'w', encoding='UTF-8') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(liste_patienten)

In [ ]:
# Liste mit Patienten mit ARDS anhand von SUBJECT_ID
liste_subject = list(df_patienten.SUBJECT_ID.unique())

In [ ]:
len(df_diagnosis[df_diagnosis.ICD9_CODE == '769'])

1314

Neugeborene mit ARDS haben eine andere Diagnose (769). Deshalb ist liste_patienten nur mit erwachsenen ARDS-Patienten gefüllt.

## drgcodes
Diagnosis Related Groups (DRG), which are used by the hospital for billing purposes. 
Bisher wissen wir nicht, wofür wir diese Informationen nutzen können.

In [ ]:
df_drgcodes = pd.read_csv('data/DRGCODES.csv.gz', 
                                compression='gzip')

In [ ]:
df_drgcodes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,342,2491,144486,HCFA,28,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",nan,nan
1,343,24958,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,nan,nan
2,344,18325,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,nan,nan
3,345,17887,182692,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,nan,nan
4,346,11113,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,nan,nan


In [ ]:
df_drgcodes.shape

(125557, 8)

In [ ]:
df_drgcodes.HADM_ID.isnull().sum()

0

In [ ]:
# ein Beispiel eines Patienten mit ARDS
df_drgcodes[df_drgcodes.HADM_ID == 110404]

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
12358,11873,268,110404,HCFA,475,RESPIRATORY SYSTEM DIAGNOSIS WITH VENTILATOR S...,nan,nan


## icustays
Every unique ICU stay in the database.  
CCU - Coronary Care Unit   
CSRU - Cardiac Surgery Recovery Unit   
MICU - Medical Intensive Care Unit  
NICU - Neonatal Intensive Care Unit   
SICU - Surgical Intensive Care Unit   
TSICU - Trauma Surgical Intensive Care Unit

In [ ]:
df_icustays = pd.read_csv('data/ICUSTAYS.csv.gz', 
                                compression='gzip')

In [ ]:
df_icustays.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2489999999999997
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.06
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [ ]:
df_icustays.HADM_ID.isnull().sum()

0

In [ ]:
df_icustays.FIRST_CAREUNIT.unique()

array(['MICU', 'CCU', 'NICU', 'TSICU', 'SICU', 'CSRU'], dtype=object)

In [ ]:
df_icustays.FIRST_WARDID.unique()

array([52, 57, 12,  7, 56, 15, 14, 23, 33, 50, 16, 22, 26, 37, 38, 47])

## Inputevent 
Inputs and outputs are extremely useful when studying intensive care unit patients. Inputs are any fluids which have been administered to the patient: such as oral or tube feedings or intravenous solutions containing medications. 
### CV 
Inputs exist in two separate tables: INPUTEVENTS_CV and INPUTEVENTS_MV. INPUTEVENTS_CV contains CareVue inputs, while INPUTEVENTS_MV contains Metavision inputs. Results from these tables can be unioned as observations are not duplicated across tables.
For CareVue data, the rate and volume will be asynchronous, and only the CHARTTIME will be available. For rates, the CHARTTIME will correspond to a start time (when the drug was set to that rate). For volumes, the CHARTTIME will correspond to an end time.


In [ ]:
df_input_cv = pd.read_csv('data/INPUTEVENTS_CV.csv.gz', 
                                compression='gzip')

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
df_input_cv.head()

In [ ]:
df_input_cv.shape

In [ ]:
df_input_cv.HADM_ID.isnull().sum()

Wir suchen von den Zeilen mit HADM_ID = NaN diejenigen heraus, wo die Subject_ID in der Liste der ARDS_Patienten ist.

In [ ]:
df_input_null= df_input_cv.loc[df_input_cv['HADM_ID'].isnull()]

In [ ]:
df_input_null_ards= df_input_null.loc[df_input_null['SUBJECT_ID'].isin(liste_subject)]

In [ ]:
df_input_null_ards.shape

We reduce the dataframe of patients with the diagnoses ARDS, based on HADM_ID.

In [ ]:
df_input_cv= df_input_cv.loc[df_input_cv['HADM_ID'].isin(liste_patienten)]

In [ ]:
df_input_cv.shape

### Inputevent MV

In [ ]:
df_input_mv = pd.read_csv('data/INPUTEVENTS_MV.csv.gz', 
                                compression='gzip')

In [ ]:
df_input_mv.head()

In [ ]:
df_input_mv.shape

In [ ]:
df_input_mv.HADM_ID.isnull().sum()

We reduce the dataframe of patients with the diagnoses ARDS, based on HADM_ID.

In [ ]:
df_input_mv= df_input_mv.loc[df_input_mv['HADM_ID'].isin(liste_patienten)]

In [ ]:
df_input_mv.shape

## Labevents
Contains all laboratory measurements for a given patient, including out patient data.
Note that the time associated with this result is the time of the fluid acquisition, not the time that the values were made available to the clinical staff.  
Some items are duplicated between the labevents and chartevents tables. In cases where there is disagreement between measurements, labevents should be taken as the ground truth.  
FLAG indicates whether the laboratory value is considered abnormal or not, using pre-defined thresholds.


In [ ]:
df_labevents = pd.read_csv('data/LABEVENTS.csv.gz', 
                                compression='gzip')

In [ ]:
df_labevents.head()

In [ ]:
df_labevents.shape

In [ ]:
# HADM_ID have NaN. Howmany?
df_labevents.HADM_ID.isnull().sum()

**MIST** Was machen wir nun?  
Wir holen die Zeilen des dataframes raus mit HADM_ID = NaN und schauen, ob Patienten (SUCJECT_ID) mit ARDS dabei sind.

In [ ]:
df_lab_null= df_labevents.loc[df_labevents['HADM_ID'].isnull()]

In [ ]:
df_lab_null.head()

In [ ]:
df_lab_null_ards= df_lab_null.loc[df_lab_null['SUBJECT_ID'].isin(liste_subject)]

In [ ]:
df_lab_null_ards.shape

1,3 Mio Labormessungen von Patienten mit ARDS sind nicht mit HADM_ID versehen.

In [ ]:
df_labevents.HADM_ID.nunique()

In [ ]:
df_labevents.SUBJECT_ID.isnull().sum()

In [ ]:
df_labevents.SUBJECT_ID.nunique()

We reduce the dataframe of patients with the diagnoses ARDS, based on HADM_ID.

## microbiologyevents
Contains microbiology information, including cultures acquired and associated sensitivities. 


In [ ]:
df_microbiology = pd.read_csv('data/MICROBIOLOGYEVENTS.csv.gz', 
                                compression='gzip')

In [ ]:
df_microbiology.head()

In [ ]:
df_microbiology.shape

In [ ]:
df_microbiology.HADM_ID.isnull().sum()

## outputevents
Output information for patients while in the ICU

In [ ]:
df_output = pd.read_csv('data/OUTPUTEVENTS.csv.gz', 
                                compression='gzip')

In [ ]:
df_output.head()

In [ ]:
df_output.shape

In [ ]:
df_output.HADM_ID.isnull().sum()

## patients 
Every unique patient in the database.

In [ ]:
df_patient = pd.read_csv('data/PATIENTS.csv.gz', 
                                compression='gzip')

In [ ]:
df_patient.head()

In [ ]:
df_patient.shape

## prescriptions
Medications ordered for a given patient.

In [ ]:
df_prescription = pd.read_csv('data/PRESCRIPTIONS.csv.gz', 
                                compression='gzip')

In [ ]:
df_prescription.head()

In [ ]:
df_prescription.shape

In [ ]:
df_prescription.HADM_ID.isnull().sum()

## procedureevents
Patient procedures for the subset of patients who were monitored in the ICU using the iMDSoft MetaVision system.

In [ ]:
df_procedure = pd.read_csv('data/PROCEDUREEVENTS_MV.csv.gz', 
                                compression='gzip')

In [ ]:
df_procedure.head()

In [ ]:
df_procedure.shape

In [ ]:
df_procedure.HADM_ID.isnull().sum()

## procedures_icd 
Patient procedures, coded using the International Statistical Classification of Diseases and Related Health Problems (ICD) system.

In [ ]:
df_procedure_icd = pd.read_csv('data/PROCEDURES_ICD.csv.gz', 
                                compression='gzip')

In [ ]:
df_procedure_icd.head()

In [ ]:
df_procedure_icd.shape

In [ ]:
df_procedure_icd.HADM_ID.isnull().sum()

In [ ]:
#How many times patients were intubated
len(df_procedure_icd[df_procedure_icd.ICD9_CODE == 9604])

## Services
Lists services that a patient was admitted/transferred under.
The services table describes the service that a patient was admitted under. While a patient can be physicially located at a given ICU type (say MICU), they are not necessarily being cared for by the team which staffs the MICU. This can happen due to a number of reasons, including bed shortage. The SERVICES table should be used if interested in identifying the type of service a patient is receiving in the hospital. For example, if interested in identifying surgical patients, the recommended method is searching for patients admitted under a surgical service.  
CMED - Cardiac Medical - for non-surgical cardiac related admissions  
CSURG - Cardiac Surgery - for surgical cardiac admissions  
DENT - Dental - for dental/jaw related admissions  
ENT - Ear, nose, and throat - conditions primarily affecting these areas  
GU - Genitourinary - reproductive organs/urinary system  
GYN - Gynecological - female reproductive systems and breasts  
MED - Medical - general service for internal medicine  
NB - Newborn - infants born at the hospital  
NBB - Newborn baby - infants born at the hospital  
NMED - Neurologic Medical - non-surgical, relating to the brain  
NSURG - Neurologic Surgical - surgical, relating to the brain  
OBS - Obstetrics - conerned with childbirth and the care of women giving birth  
ORTHO - Orthopaedic - surgical, relating to the musculoskeletal system  
OMED - Orthopaedic medicine - non-surgical, relating to musculoskeletal system  
PSURG - Plastic - restortation/reconstruction of the human body (including cosmetic or aesthetic)  
PSYCH - Psychiatric - mental disorders relating to mood, behaviour, cognition, or perceptions  
SURG - Surgical - general surgical service not classified elsewhere  
TRAUM - Trauma - injury or damage caused by physical harm from an external source  
TSURG - Thoracic Surgical - surgery on the thorax, located between the neck and the abdomen  
VSURG - Vascular Surgical - surgery relating to the circulatory system  


In [ ]:
df_services = pd.read_csv('data/SERVICES.csv.gz', 
                                compression='gzip')

In [ ]:
df_services.head()

In [ ]:
df_services.shape

In [ ]:
df_services.HADM_ID.isnull().sum()

## transfers
Patient movement from bed to bed within the hospital, including ICU admission and discharge.

In [ ]:
df_transfers = pd.read_csv('data/TRANSFERS.csv.gz', 
                                compression='gzip')

In [ ]:
df_transfers.head()

In [ ]:
df_transfers.shape

In [ ]:
df_transfers.HADM_ID.isnull().sum()

# Dictionaries

## D_CPT
 
High-level definitions for current procedural terminology (CPT) codes. Remains in the working data table because it might be useful for later explanation. 

In [ ]:
df_dcpt = pd.read_csv('data/D_CPT.csv.gz', compression='gzip')


In [ ]:
df_dcpt.head(10)

In [ ]:
df_dcpt.SECTIONHEADER.unique()

In [ ]:
df_dcpt.shape

In [ ]:
df_dcpt.SECTIONHEADER.unique()

In [ ]:
df_dcpt.SUBSECTIONHEADER.unique()

## D_ICD Diagnosis 
 
Select patients that suffer from 'acute respiratory failure' based in ICD9 Code (51881). 

In [ ]:
df_icd_diagnosis = df.read_csv('data/D_ICD_DIAGNOSES.csv.gz', 
                               compression='gzip', blocksize=None, 
                              dtype={'ICD9_CODE': 'object'})

# WATCH OUT - ICD9 is now an OBJECT

In [ ]:
df_icd_diagnosis.head()

In [ ]:
df_icd_diagnosis[dd_icd_diagnosis.ICD9_CODE == '51881']

Der IDC-9 Code für ARDS ist 51881.

In [ ]:
df_icd_diagnosis[df_icd_diagnosis.ICD9_CODE == '99591']

## D_ICD Procedures 
 
Extubation failure is defined as a re-intubation within XX time. 


In [ ]:
d_icd_procedures = df.read_csv('data/D_ICD_PROCEDURES.csv.gz', 
                               compression='gzip', blocksize=None)

In [ ]:
df_icd_procedures.head()

In [ ]:
df_icd_procedures[df_icd_procedures.SHORT_TITLE == 'Insert endotracheal tube']

## D_Items Table 
 
The D_ITEMS table does not link to the LABEVENTS table, as this data was acquired separately from the hospital database. The D_ITEMS table was acquired from the ICU databases.


##### Problem
Cant identify label for intubation

In [ ]:
df_items = df.read_csv('data/D_ITEMS.csv.gz', 
                               compression='gzip', blocksize=None, 
                       dtype={'ABBREVIATION': 'object',
                              'PARAM_TYPE': 'object',
                              'UNITNAME': 'object'})

In [ ]:
df_items.head(3)

In [ ]:
df_items[df_items.LABEL == 'endotracheal tube']

In [ ]:
df_items[df_items.ITEMID == 225792]

In [ ]:
df_items[dd_items.ITEMID == 225368] 

In [ ]:
df_items= df_items.dropna(subset=["LABEL"])

In [ ]:
df_items[df_items.LABEL.str.contains("trach")]

In [ ]:
len(df_items)

## D_Lab Items

In [ ]:
df_labitems= df.read_csv('data/D_LABITEMS.csv.gz', 
                               compression='gzip', blocksize=None)

In [ ]:
df_labitems.head()

In [ ]:
df_labitems= df.read_csv('data/D_LABITEMS.csv.gz', 
                               compression='gzip', blocksize=None)